In [1]:
import pandas as pd
import numpy as np
import networkx as nx

**Data pre-processing**

In [ ]:
df = pd.read_excel('D:\\Referral\\Referral_VIN_Prime.xls')  #Prime VINs associated with taxi
taxi_vin_list = df.REFL_VIN.tolist()

In [ ]:
vin_phone = pd.read_csv('referral_vin.csv', low_memory=False)  #Sales phone mapped using referral VINs
service_phone = pd.read_csv('refr_cust_details.csv', low_memory=False) #Service phone associated with the referral VINs

In [ ]:
# Remove Prime Taxi vins from both the above table
vin_phone = vin_phone[~vin_phone['repm_vin'].isin(taxi_vin_list)]
service_phone = service_phone[~service_phone['repm_vin'].isin(taxi_vin_list)]

In [ ]:
#Phone number pre-processing
service_phone['Phone_no'] = np.where(((service_phone['master_cell_phone'].isnull())|
                            (service_phone['master_cell_phone'].astype(str).str.startswith('1'))|
                            (service_phone['master_cell_phone'].astype(str).str.startswith('0'))), 
                            service_phone['contact_cell_phone'], service_phone['master_cell_phone'])

#Remove suspicious phone numbers
service_phone_clean = service_phone[~service_phone['Phone_no'].astype(str).str.startswith('1')]
service_phone_clean = service_phone_clean[~service_phone_clean['Phone_no'].astype(str).str.startswith('0')]
service_phone_clean = service_phone_clean[~service_phone_clean['Phone_no'].astype(str).str.len()<10]
service_phone_clean = service_phone_clean[~service_phone_clean['Phone_no'].isnull()]

#Additional list of suspicious service phone numbers
x = cust_notused.groupby(['service_phone'])['repm_vin'].nunique().reset_index().sort_values(by=['repm_vin'], ascending=False)
Bad_service_phone = x[x['repm_vin']>50].service_phone.tolist()
#Bad_service_phone list was further verified using True Caller API
#curl --request GET \
#  --url 'https://peoplestacks.com/api/v1/lookup/people/{phone_number}'
#  --header 'content-type: application/json'

service_phone_clean = service_phone_clean[~service_phone_clean['service_phone'].isin(Bad_service_phone)]

**Create unique customer group id for non-used cars VIN**

In [ ]:
#Remove used car VIN and merge service phone to referral VIN
used_car_serv = pd.read_csv('used_car_rcrepm.csv')
used_car_list = used_car_serv.repm_vin.tolist()

service_phone_notused = service_phone_clean[~service_phone_clean['repm_vin'].isin(used_car_list)][[ 
    'repm_vin', 'con_id', 'Phone_no']].rename(columns={'Phone_no': 'service_phone'})
vin_phone_notused = vin_phone[~vin_phone['repm_vin'].isin(used_car_list)][[
    'repm_vin', 'vhcl_con_id', 'phone_no']].rename(columns={'vhcl_con_id': 'con_id'})

cust_notused = pd.merge(vin_phone, service_phone_merge, how='inner', on=['repm_vin'])

In [ ]:
cust_notused['vin_grp'] = cust_notused.groupby(['repm_vin']).ngroup()+1
cust_notused['ser_ph_grp'] = cust_notused.groupby(['service_phone']).ngroup()+1
phone_vin = cust_notused.groupby(['ser_ph_grp', 'repm_vin']).count().reset_index()
phone_vin = phone_vin[['ser_ph_grp', 'repm_vin']]

nwk = pd.merge(cust_notused[['ser_ph_grp', 'repm_vin']], phone_vin, how='left', on=['ser_ph_grp'])
nwk_x = nwk.drop(['ser_ph_grp'], axis=1)

##Create network graph for service phone and VIN
G = nx.from_pandas_edgelist(nwk_x, source='repm_vin_x', target='repm_vin_y')
l = [list(x.nodes()) for x in nx.connected_component_subgraphs(G)]
d = dict((k, i) for i in range(len(l)) for k in l[i])
cust_notused['unique_cust_grp'] = cust_notused.repm_vin.map(d)
cust_notused = cust_notused[cust_notused.service_phone.notnull()]

**Create unique customer group id for used car VIN**

In [ ]:
used_car_serv['repm_ro_dtime'] = pd.to_datetime(used_car_serv['repm_ro_dtime'])
used_car_serv['used_car_sale_date'] = pd.to_datetime(used_car_serv['used_car_sale_date'].astype(str), format='%Y%m%d')

used_car_serv_2 = used_car_serv[used_car_serv['repm_ro_dtime']>=used_car_serv['used_car_sale_date']] #2nd owner of the car
used_car_serv_1 = used_car_serv[used_car_serv['repm_ro_dtime']< used_car_serv['used_car_sale_date']] #1st owner of the car

In [ ]:
used_car_serv_2 = used_car_serv_2[used_car_serv_2['repm_work_type']!='PD'] #Remove service phone associated with 'PD' type
used_car_serv_1 = used_car_serv_1[used_car_serv_1['repm_work_type']!='PD'] #Remove service phone associated with 'PD' type


In [ ]:
#Data pre-processing for used_car_serv_1. Remove bad phone numbers and merge to the vin_phone df. 
vin_phone_y = vin_phone[vin_phone['repm_vin'].isin(used_car_serv_1.repm_vin.unique().tolist())][
    ['repm_vin', 'vhcl_con_id', 'phone_no']].rename(columns={'vhcl_con_id': 'con_id'})

used_car_1_con = used_car_serv_1[~used_car_serv_1['Phone_no'].isin(Bad_service_phone)][['repm_vin', 'Phone_no']].rename(
columns={'Phone_no': 'service_phone'})
#used_car_1_con = used_car_1_con[used_car_1_con['service_phone'].notnull()]
used_car_1_con = used_car_1_con.drop_duplicates(subset=['service_phone'])

used_car_1st = pd.merge(sales_vin_y, used_car_1_con, how='left', on=['repm_vin'])
used_car_1st = used_car_1st[used_car_1st.service_phone.notnull()]


In [ ]:
#Check how many service phone numbers present in used_car_serv_1 is present in cust_notused.
#These should be assigned same customer group as present in cust_notused
notused_dict = pd.Series(cust_notused['unique_cust_grp'].values, 
                        index=cust_notused['service_phone']).to_dict()
used_car_1st_notused = used_car_1st[used_car_1st.service_phone.isin(cust_notused.service_phone.unique().tolist())]
used_car_1st_notused = used_car_1st[used_car_1st.repm_vin.isin(used_car_1st_notused.repm_vin.unique().tolist())]
used_car_1st_notused['unique_cust_grp'] = used_car_1st_notused['service_phone'].map(notused_dict)
used_1_cust_dict = pd.Series(used_car_1st_notused['unique_cust_grp'].values, 
                        index=used_car_1st_notused['repm_vin']).to_dict()
used_car_1st_notused = used_car_1st[used_car_1st.repm_vin.isin(used_car_1st_notused.repm_vin.unique().tolist())]
used_car_1st_notused['unique_cust_grp'] = used_car_1st_notused['repm_vin'].map(used_1_cust_dict)    

used_car_1st_notused['vin_grp'] = 'used_car_1st'   #Identify customer groups with used cars. Such VIN will be present across multiple customer group
used_car_1st_notused['ser_ph_grp'] = 'same_notused' #Identify customer groups with used cars. 

used_car_1st_notused = used_car_1st_notused[['repm_vin', 'con_id', 'phone_no', 'service_phone', 'vin_grp',
                                             'ser_ph_grp', 'unique_cust_grp']]
cust_grp = pd.concat([cust_notused, used_car_1st_notused]) #Concat this to cust_notused

In [ ]:
#Create unique customer group ids for used_car_1 that are not present in cust_notused
used_car_1st_unique = used_car_1st[~used_car_1st.repm_vin.isin(used_car_1st_notused.service_phone.unique().tolist())]
used_car_1st_unique['vin_grp'] = used_car_1st_unique.groupby(['repm_vin']).ngroup()+1
used_car_1st_unique['ser_ph_grp'] = used_car_1st_unique.groupby(['service_phone']).ngroup()+1

phone_vin_a = used_car_1st_unique.groupby(['ser_ph_grp', 'repm_vin']).count().reset_index()
phone_vin_a = phone_vin_a[['ser_ph_grp', 'repm_vin']]

nwk_used_1 = pd.merge(used_car_1st_unique[['ser_ph_grp', 'repm_vin']], phone_vin_a, how='left', on=['ser_ph_grp'])
nwk_used_1_x = nwk_used_1.drop(['ser_ph_grp'], axis=1)

G_used = nx.from_pandas_edgelist(nwk_used_1_x, source='repm_vin_x', target='repm_vin_y')
l = [list(x.nodes()) for x in nx.connected_component_subgraphs(G_used)]
d_used = dict((k, i) for i in range(len(l)) for k in l[i])
used_car_1st_unique['unique_cust_grp_initial'] = used_car_1st_unique.repm_vin.map(d_used)

#Add a '_used' tag to cust_grp so that the group numbers are unique
used_car_1st_unique['unique_cust_grp'] = used_car_1st_unique['unique_cust_grp_initial'].astype(str) + '_' + 'used'
used_car_1st_unique = used_car_1st_unique.drop(['unique_cust_grp_initial'], axis=1)

#Merge with cust_grp
cust_grp = pd.concat([cust_grp, used_car_1st_unique])

In [ ]:
#Used_car_2 pre-processing
used_car_2nd = used_car_serv_2[used_car_serv_2.Phone_no.notnull()][['repm_vin', 'con_id', 'master_cell_phone',
                                                                    'Phone_no']].rename(
    columns={'master_cell_phone': 'phone_no', 'Phone_no':'service_phone'})
used_car_2nd = used_car_2nd[~used_car_2nd.service_phone.isin(Bad_service_phone)]


In [ ]:
#Check how many service phone numbers present in used_car_serv_2 is present in cust_grp.
#These should be assigned same customer group as present in cust_grp
all_dict = pd.Series(cust_grp['unique_cust_grp'].values, 
                        index=cust_grp['service_phone']).to_dict()

used_car_2nd_notused = used_car_2nd[used_car_2nd.service_phone.isin(cust_grp.service_phone.unique().tolist())]
used_car_2nd_notused = used_car_2nd[used_car_2nd.repm_vin.isin(used_car_2nd_notused.repm_vin.unique().tolist())]
used_car_2nd_notused['unique_cust_grp'] = used_car_2nd_notused['service_phone'].map(all_dict)
used_2_cust_dict = pd.Series(used_car_2nd_notused['unique_cust_grp'].values, 
                        index=used_car_2nd_notused['repm_vin']).to_dict()
used_car_2nd_notused = used_car_2nd[used_car_2nd.repm_vin.isin(used_car_1st_notused.repm_vin.unique().tolist())]
used_car_2nd_notused['unique_cust_grp'] = used_car_2nd_notused['repm_vin'].map(used_1_cust_dict)    

used_car_2nd_notused['vin_grp'] = 'used_car_2nd'    #Identify customer groups with used cars.
used_car_2nd_notused['ser_ph_grp'] = 'same_notused'  #Identify customer groups with used cars.

used_car_2nd_notused = used_car_2nd_notused[['repm_vin', 'con_id', 'phone_no', 'service_phone', 'vin_grp',
                                             'ser_ph_grp', 'unique_cust_grp']]

#Merge with cust_grp
cust_grp = pd.concat([cust_grp, used_car_2nd_notused])

In [ ]:
#Create unique customer group ids for used_car_2 that are not present in cust_grp

used_car_2nd_unique = used_car_2nd[~used_car_2nd.repm_vin.isin(used_car_2nd_notused.repm_vin.unique().tolist())]
used_car_2nd_unique = used_car_2nd_unique.drop_duplicates(subset=['repm_vin', 'con_id', 'service_phone'])
used_car_2nd_unique['vin_grp'] = used_car_2nd_unique.groupby(['repm_vin']).ngroup()+1
used_car_2nd_unique['ser_ph_grp'] = used_car_2nd_unique.groupby(['service_phone']).ngroup()+1
phone_vin_b = used_car_2nd_unique.groupby(['ser_ph_grp', 'repm_vin']).count().reset_index()
phone_vin_b = phone_vin_b[['ser_ph_grp', 'repm_vin']]
nwk_used_2 = pd.merge(used_car_2nd_unique[['ser_ph_grp', 'repm_vin']], phone_vin_b, how='left', on=['ser_ph_grp'])
nwk_used_2_x = nwk_used_2.drop(['ser_ph_grp'], axis=1)

# Create the graph
G_used_2 = nx.from_pandas_edgelist(nwk_used_2_x, source='repm_vin_x', target='repm_vin_y')
l = [list(x.nodes()) for x in nx.connected_component_subgraphs(G_used_2)]
d_used_2 = dict((k, i) for i in range(len(l)) for k in l[i])

used_car_2nd_unique['unique_cust_grp_initial'] = used_car_2nd_unique.repm_vin.map(d_used_2)

#Add a '_used2' tag to cust_grp so that the group numbers are unique
used_car_2nd_unique['unique_cust_grp'] = used_car_2nd_unique['unique_cust_grp_initial'].astype(str) + '_' + 'used2'
used_car_2nd_unique = used_car_2nd_unique.drop(['unique_cust_grp_initial'], axis=1)
#Merge with cust_grp
cust_grp = pd.concat([cust_grp, used_car_2nd_unique])

#Remove duplicates
cust_grp_final = cust_grp.drop_duplicates(subset=['repm_vin', 'con_id', 'phone_no', 'service_phone'])

cust_grp.reset_index().to_json("D:\Referral\Cust_grp_master.JSON")